In [9]:
import pandas as pd

#df = pd.read_csv('C:\GitHub/rag\metrics\data.csv')
df = pd.read_csv('C:/GitHub/rag/metrics/data_temp.csv')

df

,total_predictors,community_prediction,articles,mean_ACE,question,answer,date,PlainLLM_0,PlainLLM_1,PlainLLM_2,PlainLLM_2_text
0,0.097902,0.14,"[{'id': 83, 'url': 'https://www.thelancet.com/...",0.306112,Will the CDC's assessment of the risk posed by...,1,2024-08-24,0.400,0.70,0.30,The probability is relatively low as mpox had ...
1,0.097902,0.15,"[{'id': 69, 'url': 'https://tools.cdc.gov/podc...",0.076705,Will the CDC's assessment of the risk posed by...,0,2024-08-24,0.300,0.22,0.60,Mpox cases have demonstrated spatial variabili...
2,0.090909,0.38,"[{'id': 43, 'url': 'https://menafn.com/arabic/...",0.232543,Will Greg Brockman return to OpenAI following ...,0,2024-08-24,0.250,0.65,0.30,Greg Brockman's reasons for his leave of absen...
3,0.671329,0.99,"[{'id': 99, 'url': 'https://www.cnn.com/2023/0...",0.374007,Will Robert F. Kennedy Jr. suspend his 2024 pr...,1,2024-08-24,0.205,0.35,0.32,Given Robert F. Kennedy Jr.'s past criticism o...
4,0.125874,0.54,"[{'id': 41, 'url': 'https://economictimes.indi...",0.212494,Will the Warren Buffett Indicator exceed 200% ...,0,2024-08-21,0.600,0.06,0.35,The Warren Buffett Indicator is calculated by ...
...,...,...,...,...,...,...,...,...,...,...,...
92,0.140057,0.97,"[{'id': 4, 'url': 'https://abcnews.go.com/Busi...",0.264816,Will Linda Yaccarino be the CEO of X on April ...,1,2024-05-31,0.350,0.80,0.20,Linda Yaccarino was named as the successor to ...
93,0.132867,0.65,"[{'id': 15, 'url': 'https://www.independent.co...",0.219757,Will Keir Starmer's approval rating as Prime M...,1,2024-08-17,0.720,0.52,0.58,"Based on historical trends, Labour leaders oft..."
94,0.125874,0.83,"[{'id': 17, 'url': 'https://www.irishtimes.com...",0.184166,Will Keir Starmer's approval rating as Prime M...,1,2024-08-17,0.850,0.85,0.85,Keir Starmer has not yet become Prime Minister...
95,0.132867,0.30,"[{'id': 36, 'url': 'https://news.yahoo.com/fed...",0.164507,Will the Fed hold rates steady at their Septem...,0,2024-08-23,0.675,0.48,0.62,"Historically, the Fed tends to hold rates stea..."


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   total_predictors      97 non-null     float64
 1   community_prediction  97 non-null     float64
 2   articles              97 non-null     object 
 3   mean_ACE              97 non-null     float64
 4   question              97 non-null     object 
 5   answer                97 non-null     int64  
 6   date                  97 non-null     object 
 7   PlainLLM_0            97 non-null     float64
 8   PlainLLM_1            97 non-null     float64
 9   PlainLLM_2            97 non-null     float64
 10  PlainLLM_2_text       97 non-null     object 
dtypes: float64(6), int64(1), object(4)
memory usage: 8.5+ KB


In [7]:
import os

os.chdir("../app")  # Adjust the path based on your folder structure

from rag_pipelines import AsyncBinaryQuestionPlainLLM, AsyncBinaryQuestionNaiveRAG, AsyncBinaryQuestionRAG_hybrid_crossencoder
from tqdm import tqdm
import asyncio

In [17]:
col = 'PlainLLM_2'
text = "PlainLLM_2_text"


df[col] = None
df[text] = None

model = AsyncBinaryQuestionPlainLLM()
num_retries = 3

async def process_rows():
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):

        for attempt in range(num_retries):
            try:
                anwser = await model.run(row['question'])
                df.at[index, col] = anwser.probability / 100
                df.at[index, text] = anwser.justification
                break  # Exit the retry loop if successful
            except Exception as e:
                print(f"Attempt {attempt + 1} failed for index {index}: {e}")
                if attempt == num_retries - 1:
                    df.at[index, col] = None
                    df.at[index, text] = None
                await asyncio.sleep(1)

# Run the asynchronous function
await process_rows()

Processing rows: 100%|██████████| 97/97 [04:03<00:00,  2.51s/it]


In [18]:
df

,total_predictors,community_prediction,articles,mean_ACE,question,answer,date,PlainLLM_0,PlainLLM_1,PlainLLM_2,PlainLLM_2_text
0,0.097902,0.14,"[{'id': 83, 'url': 'https://www.thelancet.com/...",0.306112,Will the CDC's assessment of the risk posed by...,1,2024-08-24,0.4,0.7,0.3,The probability is relatively low as mpox had ...
1,0.097902,0.15,"[{'id': 69, 'url': 'https://tools.cdc.gov/podc...",0.076705,Will the CDC's assessment of the risk posed by...,0,2024-08-24,0.3,0.22,0.6,Mpox cases have demonstrated spatial variabili...
2,0.090909,0.38,"[{'id': 43, 'url': 'https://menafn.com/arabic/...",0.232543,Will Greg Brockman return to OpenAI following ...,0,2024-08-24,0.25,0.65,0.3,Greg Brockman's reasons for his leave of absen...
3,0.671329,0.99,"[{'id': 99, 'url': 'https://www.cnn.com/2023/0...",0.374007,Will Robert F. Kennedy Jr. suspend his 2024 pr...,1,2024-08-24,0.205,0.35,0.32,Given Robert F. Kennedy Jr.'s past criticism o...
4,0.125874,0.54,"[{'id': 41, 'url': 'https://economictimes.indi...",0.212494,Will the Warren Buffett Indicator exceed 200% ...,0,2024-08-21,0.6,0.06,0.35,The Warren Buffett Indicator is calculated by ...
...,...,...,...,...,...,...,...,...,...,...,...
92,0.140057,0.97,"[{'id': 4, 'url': 'https://abcnews.go.com/Busi...",0.264816,Will Linda Yaccarino be the CEO of X on April ...,1,2024-05-31,0.35,0.8,0.2,Linda Yaccarino was named as the successor to ...
93,0.132867,0.65,"[{'id': 15, 'url': 'https://www.independent.co...",0.219757,Will Keir Starmer's approval rating as Prime M...,1,2024-08-17,0.72,0.52,0.58,"Based on historical trends, Labour leaders oft..."
94,0.125874,0.83,"[{'id': 17, 'url': 'https://www.irishtimes.com...",0.184166,Will Keir Starmer's approval rating as Prime M...,1,2024-08-17,0.85,0.85,0.85,Keir Starmer has not yet become Prime Minister...
95,0.132867,0.30,"[{'id': 36, 'url': 'https://news.yahoo.com/fed...",0.164507,Will the Fed hold rates steady at their Septem...,0,2024-08-23,0.675,0.48,0.62,"Historically, the Fed tends to hold rates stea..."


In [19]:
df.to_csv('C:\GitHub/rag\metrics\data_temp.csv', index=False)

In [8]:
col = 'NaiveRAG_gd_0'
text = "NaiveRAG_gd_0_text"


df[col] = None
df[text] = None

model = AsyncBinaryQuestionNaiveRAG()
num_retries = 3

async def process_rows():
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):

        for attempt in range(num_retries):
            try:
                anwser = await model.run(row['question'], date=row['date'])
                df.at[index, col] = anwser.probability / 100
                df.at[index, text] = anwser.justification
                break  # Exit the retry loop if successful
            except Exception as e:
                print(f"Attempt {attempt + 1} failed for index {index}: {e}")
                if attempt == num_retries - 1:
                    df.at[index, col] = None
                    df.at[index, text] = None
                await asyncio.sleep(1)

# Run the asynchronous function
await process_rows()

2025-04-25 23:30:54,921 | INFO | rag_pipelines.AsyncBinaryQuestionNaiveRAG | Initializing AsyncBinaryQuestionNaiveRAG with model llama-3.1-8b-instant
2025-04-25 23:30:54,922 | DEBUG | utils.helpers | Initializing AsyncGroqClient with model: llama-3.1-8b-instant
2025-04-25 23:30:55,145 | INFO | rag_pipelines.AsyncBinaryQuestionNaiveRAG | Initializing SBERT encoder
2025-04-25 23:30:55,146 | INFO | sentence_transformers.SentenceTransformer | Use pytorch device_name: cpu
2025-04-25 23:30:55,146 | INFO | sentence_transformers.SentenceTransformer | Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2025-04-25 23:30:55,369 | DEBUG | urllib3.connectionpool | https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json HTTP/1.1" 200 0
2025-04-25 23:30:55,588 | DEBUG | urllib3.connectionpool | https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json HTTP/1.1" 200 0
2025-04-25 23:30:55,814 | DE

Processing rows:   0%|          | 0/97 [00:00<?, ?it/s]

2025-04-25 23:30:57,688 | INFO | rag_pipelines.AsyncBinaryQuestionNaiveRAG | Starting async naive RAG pipeline for query: 'Will the CDC's assessment of the risk posed by mpox to the US general public exceed "Very Low" before October 1, 2024?'
2025-04-25 23:30:57,688 | INFO | rag_pipelines.AsyncBinaryQuestionNaiveRAG | Acquiring data
2025-04-25 23:30:57,690 | INFO | rag_pipelines.AsyncBinaryQuestionNaiveRAG | No data provided, fetching from The Guardian
2025-04-25 23:30:57,690 | INFO | rag_pipelines.AsyncBinaryQuestionNaiveRAG | Fetching data from The Guardian for query: 'Will the CDC's assessment of the risk posed by mpox to the US general public exceed "Very Low" before October 1, 2024?'
2025-04-25 23:30:57,690 | INFO | rag_pipelines.AsyncBinaryQuestionNaiveRAG | Extracting general topic
2025-04-25 23:30:57,692 | DEBUG | utils.helpers | Initializing GroqClient with model: llama-3.1-8b-instant
2025-04-25 23:30:57,919 | INFO | utils.helpers | Getting general topic using model: llama-3.1

Batches:   0%|          | 0/29 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/43 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-04-25 23:33:11,005 | INFO | rag_pipelines.AsyncBinaryQuestionNaiveRAG | Generated 1371 vectors
2025-04-25 23:33:11,005 | INFO | rag_pipelines.AsyncBinaryQuestionNaiveRAG | Building index
Attempt 1 failed for index 1: 'Config' object has no attribute 'index_type'
2025-04-25 23:33:12,015 | INFO | rag_pipelines.AsyncBinaryQuestionNaiveRAG | Starting async naive RAG pipeline for query: 'Will the CDC's assessment of the risk posed by mpox to the US general public exceed "Low" before October 1, 2024?'
2025-04-25 23:33:12,015 | INFO | rag_pipelines.AsyncBinaryQuestionNaiveRAG | Acquiring data
2025-04-25 23:33:12,018 | INFO | rag_pipelines.AsyncBinaryQuestionNaiveRAG | No data provided, fetching from The Guardian
2025-04-25 23:33:12,019 | INFO | rag_pipelines.AsyncBinaryQuestionNaiveRAG | Fetching data from The Guardian for query: 'Will the CDC's assessment of the risk posed by mpox to the US general public exceed "Low" before October 1, 2024?'
2025-04-25 23:33:12,019 | INFO | rag_pipelin

Processing rows:   1%|          | 1/97 [02:20<3:45:11, 140.74s/it]


CancelledError: 

In [ ]:
df